In [3]:
!pip install eli5

     |████████████████████████████████| 112kB 2.7MB/s 


In [0]:
import pandas as pd
import numpy as np
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import cross_val_score
import eli5
from eli5.sklearn import PermutationImportance
from ast import literal_eval
from tqdm import tqdm_notebook

In [8]:
cd "/content/drive/My Drive/Colab Notebooks/dw_matrix"

/content/drive/My Drive/Colab Notebooks/dw_matrix


In [0]:
df = pd.read_csv('data/men_shoes.csv', low_memory=False)

In [0]:
def run_model(feats, model = DecisionTreeRegressor(max_depth=5)):
  x = df[feats].values
  y = df['prices_amountmin'].values

  scores = cross_val_score(model, x, y, scoring='neg_mean_absolute_error')

  return np.mean(scores), np.std(scores)

In [12]:
df['brand_cat'] = df['brand'].map(lambda x: str(x).lower()).factorize()[0]
run_model(['brand_cat'])

(-58.133398968282776, 4.206122611474276)

In [25]:
model = RandomForestRegressor(max_depth=5, n_estimators=100, random_state=0)
run_model(['brand_cat'], model)

(-57.31783843165656, 4.181246596160967)

In [26]:
def parse_features(x):
  output_dict = {}
  if str(x) == 'nan': return output_dict
  features = literal_eval(x.replace('\\"', '"'))
  for item in features:
    key = item['key'].lower().strip()
    value = item['value'][0].lower().strip()
    output_dict[key] = value

  return output_dict

df['features_parsed'] = df['features'].map(parse_features)

keys = set()

df['features_parsed'].map(lambda x: keys.update(x.keys()))

len(keys)

476

In [27]:
def get_name_feat(key):
  return 'feat_' + key

for key in tqdm_notebook(keys):
  df[get_name_feat(key)] = df.features_parsed.map( lambda feats: feats[key] if key in feats else np.nan)

In [0]:
keys_stat = {}

for key in keys:
  keys_stat[key] = df[False == df[get_name_feat(key)].isnull()].shape[0] / df.shape[0] * 100

In [30]:
{k:v for k,v in keys_stat.items() if v > 30}

{'brand': 48.62691466083151,
 'color': 47.784463894967175,
 'gender': 50.17505470459519,
 'manufacturer part number': 36.252735229759296,
 'material': 34.9070021881838}

In [0]:
df['feat_brand_cat'] = df['feat_brand'].factorize()[0]
df['feat_color_cat'] = df['feat_color'].factorize()[0]
df['feat_material_cat'] = df['feat_material'].factorize()[0]
df['feat_gender_cat'] = df['feat_gender'].factorize()[0]
df['feat_sport_cat'] = df['feat_sport'].factorize()[0]
df['feat_style_cat'] = df['feat_style'].factorize()[0]
df['feat_color_cat'] = df['feat_color'].factorize()[0]
df['feat_metal type_cat'] = df['feat_metal type'].factorize()[0]
df['feat_shape_cat'] = df['feat_shape'].factorize()[0]

for key in keys:
  df[get_name_feat(key) + '_cat'] = df[get_name_feat(key)].factorize()[0]

In [31]:
df['brand'] = df['brand'].map(lambda x: str(x).lower())
df[df.brand == df.feat_brand].shape

(8846, 1002)

In [0]:
feats = ['brand_cat', 'feat_brand_cat', 'feat_shape_cat', 'feat_gender_cat', 'feat_material_cat', 'feat_style_cat', 'feat_metal type_cat', 'feat_color_cat']

model = RandomForestRegressor(max_depth=5, n_estimators=100)
result = run_model(feats, model)

In [35]:
x = df[feats].values
y = df['prices_amountmin'].values

m = RandomForestRegressor(max_depth=5, n_estimators=100, random_state=0)
m.fit(x,y)

print(result)

perm = PermutationImportance(m, random_state=1).fit(x,y)
eli5.show_weights(perm, feature_names=feats)

(-57.27881191710109, 4.286573861909598)


Weight,Feature
0.2501 ± 0.0079,brand_cat
0.0974 ± 0.0014,feat_material_cat
0.0241 ± 0.0024,feat_gender_cat
0.0172 ± 0.0005,feat_brand_cat
0.0120 ± 0.0011,feat_shape_cat
0.0085 ± 0.0009,feat_metal type_cat
0.0066 ± 0.0013,feat_color_cat
0.0042 ± 0.0010,feat_style_cat


In [36]:
ls

data/  HelloGithub.ipynb  LICENSE  matrix_one/  README.md
